In [2]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import socket
hostname = socket.gethostname()
if hostname == 'jerome-linux':
    datadir = Path('/home/jerome/Documents/GitHub')
elif hostname == 'sage2020':
    datadir = Path('/home/jiangrz/hdd23')
alphapath = datadir/Path('Data/rproc/XFe_alpha.csv')
betapath = datadir/Path('Data/rproc/logeps_beta.csv')
df_alpha = pd.read_csv(alphapath)
df_alpha = df_alpha[df_alpha != 9.99]
df_beta = pd.read_csv(betapath)
df_beta = df_beta[df_beta != 9.99]

from solar_rs import element_list
from starfit.star import SolAbu
sun = SolAbu('solas09.dat', silent=True)

In [3]:
dfel_alpha = []
dfel_beta = []
for colname in df_alpha.columns[3:]:
    pattern = r'(err)?([A-Z][a-z]?)(I)*'
    flag_err, el, state = re.match(pattern, colname).groups()
    if (el in element_list) and (flag_err is not None):
        dfel_alpha.append(el)
for colname in df_beta.columns[3:]:
    pattern = r'(err|log)_([A-Z][a-z]?)_new'
    flag_err, el = re.match(pattern, colname).groups()
    if (el in element_list) and (flag_err is not None):
        dfel_beta.append(el)
el_unique = np.unique(np.append(dfel_alpha, dfel_beta))
el_unique = np.array(element_list)[np.in1d(element_list, el_unique)]
el_names = ['%s'%el for el in el_unique]
elerr_names = ['e%s'%el for el in el_unique]

In [4]:
df_beta.loc[:, 'err_Th_new'] = np.nan
df_beta.loc[:, 'log_Th_new'] = np.nan
idx_0804 = df_beta.objname == 'J0804+5740.2016Nov'
df_beta.loc[idx_0804, 'log_Th_new'] = -1.37
df_beta.loc[idx_0804, 'err_Th_new'] = .2

In [43]:
col_prop = ['objname', 'FeH', 'eFeH']
df_combine = pd.DataFrame(columns=np.concatenate((col_prop, el_names, elerr_names)))
for colname in col_prop:
    df_combine.loc[:, colname] = df_alpha.loc[:, colname].values
df_combine.loc[:, 'objname'] = [_.split('.')[0] for _ in df_combine.loc[:, 'objname'].values]
el_read = []
eel_read = []
for colname in df_alpha.columns[3:]:
    pattern = r'(err)?([A-Z][a-z]?)(I)*'
    flag_err, el, state = re.match(pattern, colname).groups()
    col_feh = df_beta.loc[:, 'FeH']
    col_efeh = df_beta.loc[:, 'eFeH']
    if el in el_names:
        if (flag_err is not None) and (el not in eel_read):
            colname_new = 'e%s'%el
            col_elogeps = df_alpha.loc[:, colname].values
            df_combine.loc[:, colname_new] = col_elogeps
            eel_read.append(el)
        elif (flag_err is None) and (el not in el_read):
            colname_new = el
            col_xfe = df_alpha.loc[:, colname].values
            col_logeps = col_xfe + col_feh + sun.log_eps(el)
            df_combine.loc[:, colname_new] = col_logeps
            el_read = [el]
for colname in df_beta.columns[3:]:
    pattern = r'(err|log)_([A-Z][a-z]?)_new'
    flag_err, el = re.match(pattern, colname).groups()
    col_feh = df_beta.loc[:, 'FeH']
    col_efeh = df_beta.loc[:, 'eFeH']
    if flag_err is not None:
        if colname.startswith('err'):
            colname_new = 'e%s'%el
            col_elogeps = df_beta.loc[:, colname].values
            df_combine.loc[:, colname_new] = col_elogeps
        else:
            colname_new = el
            col_logeps = df_beta.loc[:, colname].values
            df_combine.loc[:, colname_new] = col_logeps
df_combine.to_csv(datadir/'Data/rproc/logeps.csv', index=False)

In [45]:
df_xfe = df_combine.copy()
col_logeps = np.array([_ in element_list for _ in df_combine.columns])
elnames = df_xfe.columns.values[col_logeps]
logeps_data = df_xfe.loc[:, col_logeps].values
feh_data = df_xfe.loc[:, 'FeH'].values.reshape(-1, 1)*np.ones(logeps_data.shape[-1])
logeps_sun = np.ones(logeps_data.shape[0]).reshape(-1, 1)*sun.log_eps(elnames)
xfe_data = logeps_data - feh_data - logeps_sun
df_xfe.loc[:, col_logeps] = xfe_data
df_xfe.to_csv(datadir/'Data/rproc/XFe.csv', index=False)

In [46]:
df_xfe.loc[17]

objname    J0804+5740
FeH             -2.47
eFeH             0.09
Li                NaN
C               -0.53
              ...    
eHo               NaN
eHf          0.114455
eOs          0.188944
ePb               NaN
eTh               0.2
Name: 17, Length: 67, dtype: object

In [54]:
for idx, row in df_xfe.iterrows():
    objname = row.objname
    objdata = np.array(row.values[3:], dtype=float)
    len_els = len(objdata)//2
    obj_el = row.keys()[3:][:len_els]
    obj_xfe = objdata[:len_els]
    obj_exfe = objdata[len_els:]
    flag_detect = ~np.isnan(obj_xfe)
    el = obj_el[flag_detect]
    xfe = obj_xfe[flag_detect]
    exfe = obj_exfe[flag_detect]

    stardir = datadir / 'Data/rproc/star'
    starpath = stardir / ('%s.dat'%objname)
    with open(starpath, 'w') as file:
        file.write('10002\n')
        file.write('%s\n'%(objname))
        file.write('%s\n'%(''))
        file.write('%s\n'%(''))
        file.write('3\n')
        file.write('Fe\n')
        file.write('%s\n'%(len(xfe)))
        file.write('%-3s  %5.2f  %5.2f\n'%('Fe', -2.47, 0.09))
        for _el, _xfe, _exfe in zip(el, xfe, exfe):
            if _el == 'Fe':
                continue
            el_line = '%-3s  %5.2f  %5.2f'%(_el, _xfe, _exfe)
            file.write('%s\n'%(el_line))
        file.write('As09')